In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm.notebook import tqdm
import re

In [150]:
df = pd.read_csv('test.csv')

In [151]:
def del_comment(content):
    new_content = ""
    i = 0
    while i < len(content):
        if content[i:i+2] == '/*':
            i += 2
            while i < len(content) and content[i:i+2] != '*/':
                i += 1
            i += 2  # 블록 주석의 끝 부분 넘어가기
            continue
        if content[i:i+2] == '//':
            i += 2
            while i < len(content) and content[i] != '\n':
                i += 1
            continue
        new_content += content[i]
        i += 1

    # 한 줄 주석 제거 후 남은 라인들을 다시 조합
    new_lines = [line for line in new_content.split('\n') if line.strip() != '']

    code = ' '.join(new_lines)
    return code

In [152]:
def del_function(text):
    return re.sub('\s', ' ',re.sub('\t',' ',re.sub('\n',' ',text)))

def only_ascii(text):
    return text.encode('ascii', 'ignore').decode()

df['code1'] = df['code1'].apply(del_comment)
df['code2'] = df['code2'].apply(del_comment)
df = df.applymap(del_function)
#df['code1'] = df['code1'].apply(only_ascii)
#df['code2'] = df['code2'].apply(only_ascii)

In [156]:
df.to_csv('test_final.csv',index=False)

In [157]:
df['code1'] = df['code1'].apply(only_ascii)
df['code2'] = df['code2'].apply(only_ascii)

In [158]:
df.to_csv('test_only_ascii_final.csv',index=False)

In [161]:
for i in df['code2']:
    if(i.find("/*")!=-1):
        print(1)

In [162]:
for i in df['code2']:
    if(i.find("//")!=-1):
        print(1)

In [131]:
import os
import glob
import re

def read_cpp_codes_from_folders(root_dir):
    """
    지정된 루트 디렉토리 아래의 모든 폴더에서 .cpp 파일들을 읽어서
    {라벨: [파일1의 내용, 파일2의 내용, ...]} 형태의 딕셔너리로 반환합니다.
    폴더 이름에서 숫자 부분을 라벨로 사용합니다.
    """
    label_to_code = {}
    
    # 각 라벨(폴더)를 순회합니다.
    for folder in tqdm(os.listdir(root_dir)):
        folder_path = os.path.join(root_dir, folder)
        if os.path.isdir(folder_path):
            # 폴더 이름에서 숫자 부분(라벨)을 추출합니다.
            label_match = re.search(r'\d+', folder)
            if label_match:
                label = int(label_match.group(0))
                cpp_files = glob.glob(os.path.join(folder_path, '*.cpp'))
                code_list = []
                
                # 각 .cpp 파일을 열고 내용을 읽습니다.
                for cpp_file in cpp_files:
                    with open(cpp_file, 'r', encoding='utf-8') as file:
                        content = file.read()
                            
                        new_content = ""
                        i = 0
                        while i < len(content):
                            if content[i:i+2] == '/*':
                                i += 2
                                while i < len(content) and content[i:i+2] != '*/':
                                    i += 1
                                i += 2  # 블록 주석의 끝 부분 넘어가기
                                continue
                            if content[i:i+2] == '//':
                                i += 2
                                while i < len(content) and content[i] != '\n':
                                    i += 1
                                continue
                            new_content += content[i]
                            i += 1

                        # 한 줄 주석 제거 후 남은 라인들을 다시 조합
                        new_lines = [line for line in new_content.split('\n') if line.strip() != '']

                        code = ' '.join(new_lines)

                        code = re.sub('\n',' ',code)
                        code = re.sub('\t',' ',code)
                        code = re.sub('\s',' ',code)
                        #code = code.encode('ascii', 'ignore').decode()
                        #code = code.lower()
                        code_list.append(code)
                
                # 추출한 라벨을 키로 하여 코드 리스트를 저장합니다.
                if label not in label_to_code:
                    label_to_code[label] = code_list
                else:
                    label_to_code[label].extend(code_list)
            
    return label_to_code

# 루트 디렉토리 경로를 설정하세요.
root_dir = '../train_code'
cpp_codes = read_cpp_codes_from_folders(root_dir)

# 결과를 확인합니다.
for label, codes in cpp_codes.items():
    print(f"Label: {label}, Number of CPP files: {len(codes)}")


  0%|          | 0/500 [00:00<?, ?it/s]

Label: 156, Number of CPP files: 500
Label: 317, Number of CPP files: 500
Label: 61, Number of CPP files: 500
Label: 390, Number of CPP files: 500
Label: 171, Number of CPP files: 500
Label: 454, Number of CPP files: 500
Label: 199, Number of CPP files: 500
Label: 200, Number of CPP files: 500
Label: 195, Number of CPP files: 500
Label: 361, Number of CPP files: 500
Label: 452, Number of CPP files: 500
Label: 403, Number of CPP files: 500
Label: 388, Number of CPP files: 500
Label: 301, Number of CPP files: 500
Label: 99, Number of CPP files: 500
Label: 98, Number of CPP files: 500
Label: 230, Number of CPP files: 500
Label: 492, Number of CPP files: 500
Label: 401, Number of CPP files: 500
Label: 269, Number of CPP files: 500
Label: 475, Number of CPP files: 500
Label: 194, Number of CPP files: 500
Label: 192, Number of CPP files: 500
Label: 351, Number of CPP files: 500
Label: 486, Number of CPP files: 500
Label: 107, Number of CPP files: 500
Label: 467, Number of CPP files: 500
Labe

In [132]:
import pandas as pd


# 정수형으로 키 변환
cpp_codes_dict_int_keys = {int(key): value for key, value in cpp_codes.items()}

# DataFrame 생성을 위한 데이터 준비
data = []
for label, codes in cpp_codes_dict_int_keys.items():
    for code in codes:
        data.append({'label': label, 'code': code})

# DataFrame 생성
df = pd.DataFrame(data)
df = df.sort_values(by=['label'], axis=0)
df['label'] = df['label'].apply(lambda x: x -1)
df.head()  # DataFrame의 처음 몇 줄을 출력하여 확인


,label,code
50408,0,#include<iostream> #include<cstdlib> #include<...
50409,0,#define _USE_MATH_DEFINES #include <iostream> ...
50474,0,#include <iostream> using namespace std; using...
50475,0,#include <stdio.h> #include <iostream> int mai...
50476,0,"#include<bits/stdc++.h> #define rep(i,n) for (..."


In [171]:
df = pd.read_csv('test_only_ascii_final.csv')

In [172]:
for i in df['code1']:
    if(i.find("/*")!=-1):
        print(1)

In [173]:
for i in df['code1']:
    if(i.find("/*")!=-1):
        print(1)

In [3]:
df1 = pd.read_csv('../unix_soft.csv')
df2 = pd.read_csv('../unix_soft_last.csv')

In [4]:
np.sum(df1['similar']==df2['similar'])

594434